In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 51 kB 185 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=c9946368b247f0ea53006c6a542c99be1f92e198e0e09f3e4a8c9fa1c6b67195
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import anvil.server

In [ ]:
anvil.server.connect("U7ZZGDPBK4BJXCYFDKUEPOBZ-YP6JAXX244DEFQS6")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [ ]:
dataset=pd.read_excel("/content/drive/MyDrive/Tarp data/BG_Food.xlsx", sheet_name=0)
df = pd.read_excel("/content/drive/MyDrive/Tarp data/diab.xlsx", sheet_name=0)
food_Ysuit = set()
food_Nsuit = set()
food_Csuit = set()

In [ ]:
df.head()

,Age,F,M,AB,B,O,A,High BP,Low BP,Normal,bmi,Diabeties
0,23.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,28.0,0.0
1,22.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,24.0,1.0
2,22.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,27.0,1.0
3,26.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,21.0,1.0
4,24.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,20.0,1.0


In [ ]:
def checkbg(bg):
    if(bg=="A"):
        outy='GROUP A'
    elif(bg=="B"):
        outy='GROUP B'
    elif(bg=="O"):
        outy='GROUP O'
    elif(bg=="AB"):
        outy='GROUP AB'
    food_Ysuit.update(set(list(dataset[dataset[outy]=="H"]["FOOD"])))
    food_Nsuit.update(set(list(dataset[dataset[outy]=="L"]["FOOD"])))
    food_Csuit.update(set(list(dataset[dataset[outy]=="M"]["FOOD"])))

In [ ]:
def checkbp(bp):
    if(bp.lower()=="high bp"):
        food_Ysuit.intersection_update(set(dataset[dataset["BP_EAT"]=="Y"]["FOOD"]))
        food_Nsuit.union(set(dataset[dataset["BP_EAT"]=="N"]["FOOD"]))
        food_Csuit.union(set(dataset[dataset["BP_EAT"]=="C"]["FOOD"]))
    elif(bp.lower()=="low bp"):
        food_Ysuit.intersection_update(set(dataset[dataset["BP_EAT"]=="N"]["FOOD"]))
        food_Csuit.union(set(dataset[dataset["BP_EAT"]!="N"]["FOOD"]))
    elif(bp.lower()=="normal"):
        food_Ysuit.intersection_update(set(dataset["FOOD"].dropna()))

In [ ]:
import sklearn
x = df.iloc[:,:-1]
y = df.iloc[:,12:13]
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.25,random_state=0)
def LogReg():
    from sklearn.linear_model import LogisticRegression
    clf=LogisticRegression()
    clf.fit(xtrain,ytrain)
    y_pred=clf.predict(xtest)
    from sklearn.metrics import accuracy_score
    acc_log=accuracy_score(y_pred, ytest)*100
    from sklearn.metrics import confusion_matrix
    c3=confusion_matrix(ytest,y_pred)
    print(c3)
    return acc_log

In [ ]:
def checkdia(dia):
    if(dia.lower()=="yes"):
        food_Ysuit.intersection_update(set(dataset[dataset["DIA_EAT"]=="Y"]["FOOD"]))
        food_Nsuit.union(set(dataset[dataset["DIA_EAT"]=="N"]["FOOD"]))
        food_Csuit.union(set(dataset[dataset["DIA_EAT"]=="C"]["FOOD"]))
    elif(dia.lower()=="no"):
        food_Ysuit.intersection_update(set(dataset["FOOD"]))

In [ ]:
def checkweight(weight):
    weight=int(weight)
    if(weight<=24.9 and weight>=18.5):
        food_Ysuit.intersection_update(set(dataset["FOOD"]))
    elif(weight>24.9):
        food_Ysuit.intersection_update(set(dataset[dataset["HB"]=="Y"]["FOOD"]))
        food_Nsuit.union(set(dataset[dataset["HB"]=="N"]["FOOD"]))
        food_Csuit.union(set(dataset[dataset["HB"]=="C"]["FOOD"]))
    elif(weight<18.5):
        food_Ysuit.intersection_update(set(dataset[dataset["HB"]!="Y"]["FOOD"]))
        food_Nsuit.union(set(dataset[dataset["HB"]=="Y"]["FOOD"]))
    else:
        return "input error retype the input weight(in kgs)"
    return ""

In [ ]:
@anvil.server.callable
def predict_answer(BP,BG,Dia,Weight):
    checkbg(BG)
    checkbp(BP)
    checkdia(Dia)
    r4 = checkweight(Weight)
  #checkweight(Weight)
    a1=" food that suits you--->"+", ".join(list(food_Ysuit))
    a2=" food that should be taken with caution/limited---->"+", ".join(list(food_Csuit))
    a3=" food that should be avoided---->"+", ".join(list(food_Nsuit))
    return "The suitable diet for you is"+a1+"\n\n"+a2+"\n\n"+a3
anvil.server.wait_forever()

In [ ]:
" food that suits you--->"+", ".join(list(food_Ysuit))

In [ ]:
print(predict_answer("low bp","AB","yes",32))

In [ ]:
print("helllo welcome to \"the best food that suits you\"!!")
print("we need the following data for the validation and analysis")
print("your sweet name pls  :")
name.append(((input().casefold()).capitalize()))
print("blood group(AB/O/A/B)")
bg.append(input().upper())
print("blood pressure presence(hBP/lowBP/norml)")
bp.append(input().lower())
print("diabetes presence(y/n)")
dia.append(input())
print("weight(in kgs)")
try:
    def wi():
        weight.append(int(input()))
    wi()
except:
    print("retype the input")
print("the best food that suits you are")
checkbg()
checkbp()
checkdia()
checkweight()

In [ ]:
import sklearn
import random
x = df.iloc[:,:-1]
y = df.iloc[:,12:13]
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.001,random_state=1)

In [ ]:
@anvil.server.callable
def predict_dia(BP,BG,BMI,Age,Gender):
  A = (BG =="A")
  AB = (BG =="AB")
  B = (BG =="B")
  O = (BG =="O")
  age =int(Age)
  high_bp = (BP == "High BP")
  low_bp = (BP == "Low BP")
  normal = (BP== "Normal")
  bmi = BMI
  Male = (Gender=="Male")
  Female =(Gender =="Female")
  return random.randint(0,1)
anvil.server.wait_forever()

In [ ]:
xtrain.head()
 from sklearn.linear_model import LogisticRegression
  clf=LogisticRegression()
  clf.fit(xtrain,ytrain)
  a =np.array([age,Female,Male,AB,B,O,A,high_bp,low_bp,bmi]).reshape(1,-1)
  y_pred=clf.predict(a)
  return y_pred

In [ ]:
print(predict_dia("High BP","A",20,50,"Male"))